In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from urllib import request
from urllib.request import urlretrieve
import librosa
from librosa import display
import matplotlib.pyplot as plt
import skimage
from shutil import copy2
import random
from pathlib import Path

In [ ]:
track_filepath = os.path.join(sys.path[0], "metal_subgenre_data.csv")
df = pd.read_csv(track_filepath)
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['index']

In [ ]:
df['genre'].unique()

In [ ]:
#convert mp3 to a melspectrogram and save as png

mp3_folder = os.path.join(sys.path[0], "metal_track_previews/")
spectrogram_folder = os.path.join(sys.path[0], "/metal_spectrograms/")

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

i = 0
for index, row in df.iterrows():        
    track_id = str(row['track_id'])
    prefix = track_id[0:2] + "/"
    full_directory = spectrogram_folder + prefix
    
    if not os.path.exists(full_directory + track_id + ".png"):
  
        try:
            #make a spectrogram from an mp3 file
            file_name = mp3_folder + prefix + track_id + ".mp3"
            y, sr = librosa.load(file_name)
            spect = librosa.feature.melspectrogram(y=y, sr=sr)
            spect = np.log(spect + 1e-9)
            spect = scale_minmax(spect, 0, 255).astype(np.uint8)
            spect = np.flip(spect, axis=0) #low freq. at the bottom
            spect = 255 - spect

            #save the spectrogram in the relevant sub-folder
            if not os.path.isdir(full_directory):
                os.makedirs(full_directory)

            skimage.io.imsave(full_directory + track_id + ".png", spect) 

            i += 1
            if i % 1000 == 0:
                print(str(i) + " spectrograms made.")
                
        except Exception as e:
            print(e)
            print("Skipping track.")

In [ ]:
df['genre'].value_counts()